In [8]:
import warnings
warnings.filterwarnings(action="ignore")
import torch
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision.datasets import FashionMNIST
from torchvision.transforms import transforms

In [9]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,))
])

In [10]:
train_dataset = FashionMNIST(root=".", train=True, download=True, transform=transform)
test_dataset = FashionMNIST(root=".", train=False, download=True, transform=transform)

100%|██████████████████████████████████████████████████████████████████| 26421880/26421880 [00:28<00:00, 932311.10it/s]


Extracting .\FashionMNIST\raw\train-images-idx3-ubyte.gz to .\FashionMNIST\raw



100%|████████████████████████████████████████████████████████████████████████| 29515/29515 [00:00<00:00, 142471.30it/s]


Extracting .\FashionMNIST\raw\train-labels-idx1-ubyte.gz to .\FashionMNIST\raw



100%|████████████████████████████████████████████████████████████████████| 4422102/4422102 [00:05<00:00, 792417.60it/s]


Extracting .\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to .\FashionMNIST\raw



100%|█████████████████████████████████████████████████████████████████████████| 5148/5148 [00:00<00:00, 4012688.53it/s]


Extracting .\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to .\FashionMNIST\raw



In [11]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [12]:
classes = ["T-shirt/Top", "Trouser", "Pullover", "Dress", "Coat",  "Sandal",  "Shirt", "Sneaker", "Bag", "Ankle Boot"]

In [13]:
device = torch.device('mps')

In [14]:
class Dimension():
    def __init__(self, size, RGB=False):
        self.__size:int = size
        self.__in_channels:int = 3 if RGB else 1
        self.printDimension('Inital tensor size')

    def printDimension(self, text="Tensor size:"):
        print(f"{text}: {self.__size}x{self.__size}x{self.__in_channels}")
   
    def Convolution(self, out_channels:int, kernel_size:int, stride=1, padding=0):
        self.__in_channels = out_channels
        self.__size = ((self.__size - kernel_size + 2*padding) // stride) + 1
        self.printDimension('After convolution')
        return self
   
    def Pooling(self, kernel_size:int):
        self.__size = self.__size // kernel_size
        self.printDimension('After pooling')
        return self


In [15]:
image_size = 28 
Prepocessing = Dimension(size=image_size, RGB=False)
Prepocessing.Convolution(32, kernel_size=5).Pooling(2).Convolution(64, kernel_size=5).Pooling(2)

Inital tensor size: 28x28x1
After convolution: 24x24x32
After pooling: 12x12x32
After convolution: 8x8x64
After pooling: 4x4x64


In [23]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
            
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5)
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5)
        self.pool2 = nn.MaxPool2d(2)

        self.flatten = nn.Flatten() 
        self.fc1 = nn.Linear(in_features=4*4*64, out_features=256)
        self.drop1 = nn.Dropout(p=0.3)
        self.fc2 = nn.Linear(in_features=256, out_features=64)
        self.drop2 = nn.Dropout(p=0.3)
        self.fc3 = nn.Linear(in_features=64, out_features=10)
      
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)

        x = F.relu(self.conv2(x))
        x = self.pool2(x)

        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = self.drop1(x)
        x = F.relu(self.fc2(x))
        x = self.drop2(x)
        x = F.relu(self.fc3(x))
        x = F.log_softmax(x, dim=1)
        return x

In [ ]:
model = ConvNet()
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0

    for inputs, targets in tqdm(train_loader):
        inputs = inputs.to(device)
        targets = targets.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    train_loss /= len(train_loader)

    model.eval()
    test_loss = 0.0
    test_correct = 0

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()

    test_loss /= len(test_loader)

    test_accuracy = 100.0 - test_loss

    print(f"Epoch [{epoch+1}/{num_epochs}], "
          f"Train Loss: {train_loss:.4f}, "
          f"Test Accuracy: {test_accuracy:.2f}%")